<a href="https://colab.research.google.com/github/boar1ang/big-data-challenge/blob/master/Amzn_Books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-15 23:14:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-05-15 23:14:14 (1.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [0]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amzn_Books").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles, SQLContext
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
books_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), sep="\t", header=True)


# Show DataFrame
books_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

Count & clean up

In [5]:
print(books_df.count())
books_df = books_df.dropna()
print(books_df.count())
books_df = books_df.dropDuplicates()
print(books_df.count())

10319090
10316622
10316622


Reduce DataFrame by dropping unnecessary columns & splitting 'review' data off from 'product' data

In [6]:
# Load sql column function
from pyspark.sql.functions import col

# Filter for only verified purchases
books_df = books_df.filter(col("verified_purchase")  == 'Y')
books_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35531538|R10086W0U9OHY1|1421695103|     828616333|Basset Hound Pupp...|           Books|          5|            0|          0|   N|                Y| A true Basset lover|For the seventh y...| 2012-12-28|
|         US|   18451121|R100OI6P955T8N|0764585924|     843831426|CliffsNotes on Sh...|           Books|          5|    

In [7]:
print(books_df.count())

7449299


Cleaning and stripping off unverified purchase reviews resulted in 2,869,791 fewer rows.

In [8]:
ver_BookReviews_df = books_df.select(["customer_id", "review_id", "product_id", "product_parent", "product_title", "product_category", "star_rating", "helpful_votes", "total_votes", "vine", "review_headline", "review_body", "review_date" ])
ver_BookReviews_df.show(5)

+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+--------------------+--------------------+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|     review_headline|         review_body|review_date|
+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+--------------------+--------------------+-----------+
|   35531538|R10086W0U9OHY1|1421695103|     828616333|Basset Hound Pupp...|           Books|          5|            0|          0|   N| A true Basset lover|For the seventh y...| 2012-12-28|
|   18451121|R100OI6P955T8N|0764585924|     843831426|CliffsNotes on Sh...|           Books|          5|            1|          1|   N|         Well Needed|These notes were ...| 2014-05-12|
|   27809831|R100QVQSCIC23H|1450252001|     483095

In [9]:
ver_BookReviews_df.count()

7449299

Create 'ver_reviews_df' to match 'reviews' table in Postgres database

In [10]:
ver_reviews_df = ver_BookReviews_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "review_headline", "review_body", "review_date", "customer_id"])
ver_reviews_df.show(5)

+--------------+-----------+-------------+-----------+----+--------------------+--------------------+-----------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|     review_headline|         review_body|review_date|customer_id|
+--------------+-----------+-------------+-----------+----+--------------------+--------------------+-----------+-----------+
|R10086W0U9OHY1|          5|            0|          0|   N| A true Basset lover|For the seventh y...| 2012-12-28|   35531538|
|R100OI6P955T8N|          5|            1|          1|   N|         Well Needed|These notes were ...| 2014-05-12|   18451121|
|R100QVQSCIC23H|          3|            0|          0|   N| Interesting Story..|Have always been ...| 2014-02-11|   27809831|
|R100REDE8CZG1Z|          5|            2|          3|   N|"My" standard review|The relative stor...| 2013-06-20|   13544187|
|R100U5WN59IKMO|          5|            1|          1|   N|          Great Deal|I love Louise Hay...| 2015-07-03|   38

In [11]:
products_reviewed_df = ver_BookReviews_df.select(["product_id", "product_parent", "product_title", "product_category", "review_id"])
products_reviewed_df.show(5)

+----------+--------------+--------------------+----------------+--------------+
|product_id|product_parent|       product_title|product_category|     review_id|
+----------+--------------+--------------------+----------------+--------------+
|1421695103|     828616333|Basset Hound Pupp...|           Books|R10086W0U9OHY1|
|0764585924|     843831426|CliffsNotes on Sh...|           Books|R100OI6P955T8N|
|1450252001|     483095575|For the Tsar and ...|           Books|R100QVQSCIC23H|
|0955176247|     348681061|Awake in the Hear...|           Books|R100REDE8CZG1Z|
|1561706124|     436984722|Power Thought Car...|           Books|R100U5WN59IKMO|
+----------+--------------+--------------------+----------------+--------------+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import countDistinct
unique_books = products_reviewed_df.agg(countDistinct("product_id")).collect()
unique_books

[Row(count(DISTINCT product_id)=1833795)]

Count of unique books reviewed = 1,833,795

Postgres Setup & RDS Configuration

In [0]:
mode = "append"
jdbc_url = "jdbc:postgresql://myaws-postgres.cg43u4ybmrtw.us-east-1.rds.amazonaws.com:5432/my_amzn_reviews_db"
config = {
    "user":"root",
    "password":"",
    "driver":"org.postgresql.Driver"
}

Write dataframes to target tables in my_amzn_reviews_db


In [0]:
ver_reviews_df.write.jdbc(url=jdbc_url, table="reviews", mode=mode, properties=config)

In [0]:
products_reviewed_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)